# Dia 9 | Pré-processamento: Normalização, Divisão e Limpeza de Dados

Neste notebook vamos estudar as etapas fundamentais de **pré-processamento de dados** em Machine Learning.
Abordaremos:
- Limpeza de dados (missing values, outliers, inconsistências)
- Normalização (StandardScaler / Z-score)
- Divisão em treino e teste (train_test_split)

Utilizaremos o dataset **Iris** como exemplo prático.

## 1️⃣ Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

## 2️⃣ Carregando o dataset Iris

In [2]:
# Carregar dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Visualizar as primeiras linhas
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 🔹 Explicação
- `df` contém as features do Iris e a coluna `target` como variável alvo.
- Esse dataset será utilizado para demonstrar técnicas de pré-processamento.

## 3️⃣ Normalização dos Dados

In [3]:
# Separar features (somente colunas numéricas)
X = df[iris.feature_names]  # features

# Aplicar StandardScaler (Z-score)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Transformar de volta em DataFrame para melhor visualização
df_scaled = pd.DataFrame(X_scaled, columns=iris.feature_names)

print("=== Antes da normalização ===")
display(df.head())

print("=== Depois da normalização (Z-score) ===")
display(df_scaled.head())

=== Antes da normalização ===


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


=== Depois da normalização (Z-score) ===


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


### 🔹 Explicação
- Normalização transforma os dados para **média 0 e desvio padrão 1**, evitando que features com escalas maiores dominem o modelo.
- Essencial para algoritmos que dependem de distância ou gradiente, como **Logistic Regression**, **SVM** e **KNN**.

## 4️⃣ Divisão em Treino e Teste

In [4]:
# Definir features e target
X = df_scaled
y = df['target']

# Divisão em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

### 🔹 Explicação
- `train_test_split` garante que temos conjuntos separados para treinar e avaliar o modelo.
- `random_state` permite **reprodutibilidade** dos resultados.

## 5️⃣ Checagem de dados faltantes e limpeza

In [5]:
# Verificar valores ausentes
df.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

### 🔹 Explicação
- Nenhum valor ausente neste dataset, mas em bases reais, podemos preencher (`impute`) com média, mediana ou modo, ou remover linhas/colunas.

## 6️⃣ Outliers (Exploração rápida)

In [6]:
from scipy import stats
z_scores = np.abs(stats.zscore(X_train))
outlier_mask = (z_scores > 3).any(axis=1)
outlier_rate = outlier_mask.mean()
print('Percentual de outliers:', round(outlier_rate*100, 2), '%')

Percentual de outliers: 0.0 %


### 🔹 Explicação
- Usamos Z-score > 3 como regra prática para identificar outliers.
- Em datasets reais, podemos **capping** ou **winsorizing** para limitar extremos, mas com cuidado para não distorcer os dados.

## 7️⃣ Treinando um modelo de exemplo

In [7]:
# Modelo de Regressão Logística
log_reg = LogisticRegression(max_iter=200, random_state=42)
log_reg.fit(X_train, y_train)

# Predição
y_pred_log = log_reg.predict(X_test)

# Avaliação
print('🔹 Regressão Logística')
print('Acurácia:', round(accuracy_score(y_test, y_pred_log),4))
print(classification_report(y_test, y_pred_log, target_names=iris.target_names))

🔹 Regressão Logística
Acurácia: 1.0
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      1.00      1.00        13
   virginica       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### 🔹 Explicação
- Avaliamos o modelo com **accuracy** e **classification report**.
- O modelo aprende padrões após pré-processamento e normalização.